In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import pickle
from gensim.models import Word2Vec

In [ ]:
data1 = pickle.load(open('/content/drive/MyDrive/Minor Project/data1.pkl', 'rb'))

In [ ]:
data1['Frequency'] = data1.groupby('UserId')['UserId'].transform('count')
data1.sort_values('Frequency', inplace=True, ascending=False)

In [ ]:
data = data1[:6000]

In [ ]:
words = list(data['ItemId'].unique())

In [ ]:
def prepare_dataset(window, top_n, words, data):
   
    df = data.groupby(by=['SessionId'])
    for s, d in df:
        d = d.sort_values(by=['Time'])
    
    basket = []
    next_song = []
    for s, d in df:
        words = list(d['ItemId'])
        i = 1
        j = top_n
        m = window+top_n
        if m > len(d):
            m = len(d)
        while j < m:
            basket.append(words[0:i])
            next_song.append(words[i:j])
            i += 1
            j += 1
        for i in range(len(d)-window-top_n-1):
            basket.append(words[i:i+window])
            next_song.append(words[i+window:i+window+top_n])

    return (basket, next_song)

In [ ]:
def get_dataset(window, top_n, words, data, m, emb_length):
    
    (basket, next_song) = prepare_dataset(window, top_n, words, data)
            
    song_basket = np.zeros((len(basket), window, emb_length), dtype=np.int32)
    recommended_song = np.zeros((len(next_song), top_n), dtype=np.int32)
    for i, each_words in enumerate(basket):
       for j, each_word in enumerate(each_words):
          emb = m[each_word]
          for k in range(emb_length):
            song_basket[i, j, k] = emb[k]
    
    for i, top_songs in enumerate(next_song):
        for j, song  in enumerate(top_songs):
            recommended_song[i, j] = song
    
    return (song_basket, recommended_song)

In [ ]:
(basket, next_song) = prepare_dataset(5, 1, words, data)

In [ ]:
lst = []
for i in range(len(basket)):
  lst.append(basket[i]+next_song[i])

In [ ]:
exc = []
for w in words:
  exc.append(w)
  for l in lst:
    if w in l:
      exc.pop()
      break;

In [ ]:
lst.append(exc)

In [ ]:
model = Word2Vec(sentences=lst, min_count=1, vector_size=100)

In [ ]:
m = {}
x = list(words)
for i, w in enumerate(x):
  m[w] = model.wv.get_vector(w, norm=True)

In [ ]:
pickle.dump(m, open('/content/drive/MyDrive/Minor Project/w2v_embs_100.pkl', 'wb'))

In [ ]:
(song_basket, recommended_song) = get_dataset(5, 1, words, data, m, 50)

In [ ]:
pickle.dump(song_basket, open('/content/drive/MyDrive/Minor Project/song_basket_w2v_5_1_50.pkl', 'wb'))
pickle.dump(recommended_song, open('/content/drive/MyDrive/Minor Project/recommended_song_w2v_5_1_50.pkl', 'wb'))

In [ ]:
(song_basket, recommended_song) = get_dataset(5, 1, words, data, m, 100)

In [ ]:
pickle.dump(song_basket, open('/content/drive/MyDrive/Minor Project/song_basket_w2v_5_1_100.pkl', 'wb'))
pickle.dump(recommended_song, open('/content/drive/MyDrive/Minor Project/recommended_song_w2v_5_1_100.pkl', 'wb'))

In [ ]:
(basket, next_song) = prepare_dataset(5, 1, words, data)

In [ ]:
lst = []
for i in range(len(basket)):
  lst.append(basket[i]+next_song[i])

In [ ]:
exc = []
for w in words:
  exc.append(w)
  for l in lst:
    if w in l:
      exc.pop()
      break;

In [ ]:
lst.append(exc)

In [ ]:
model = Word2Vec(sentences=lst, min_count=1, vector_size=150)

In [ ]:
m = {}
x = list(words)
for i, w in enumerate(x):
  m[w] = model.wv.get_vector(w, norm=True)

In [ ]:
pickle.dump(m, open('/content/drive/MyDrive/Minor Project/w2v_embs_150.pkl', 'wb'))

In [ ]:
(song_basket, recommended_song) = get_dataset(5, 1, words, data, m, 150)